## DT8060
## Raffaello Baluyot

# Local Model-Agnostic
In this lab, we will try and explain a models output using local model-agnostic techniques.
Libraries SHAP and LIME will be introduced and used on models that have been trained on the [Wine Quality dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality).
The two models we use are logistic regression form scikit-learn and a gradient boosted classification trees using xgboost.

The student is then to apply the methods and try to explain a black box model's output and identify whether improvements can be made for the explainability of the model.

## Load packages

In [ ]:
import pandas as pd
import numpy as np

import shap
import lime
import time
import xgboost

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

## Load the dataset and process

In [ ]:
wine = pd.read_csv('../data/winequality-red.csv', sep=';')
wine.head()

The **quality** column is based on a sensory test ranging from 1 to 10, where a higher value is considered to be a better wine.
We remake that column to be a binary classification problem, where 0 is a lower quality wine and 1 is a higher quality wine.

In [ ]:
wine['quality'] = np.where(wine['quality'] > 5, 1, 0)
wine.head()

Divide the dataset and train the two models.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(wine.iloc[:,:11].values, wine['quality'], test_size=0.3, random_state=42)
lr = LogisticRegression(max_iter=1000)
xgb = xgboost.XGBClassifier(n_estimators=500, max_depth=2)
lr.fit(X_train, Y_train)
xgb.fit(X_train, Y_train)
print(accuracy_score(lr.predict(X_test), Y_test))
print(accuracy_score(xgb.predict(X_test), Y_test))

## SHAP
Calculate the shap values.

In [ ]:
mask = shap.maskers.Independent(X_train)
lr_explainer = shap.LinearExplainer(lr, mask, feature_names=wine.columns[:11])
lr_shap_values = lr_explainer.shap_values(X_test)
lr_shaps = lr_explainer(X_test)

xgb_explainer = shap.TreeExplainer(xgb, X_train, feature_names=wine.columns[:11])
xgb_shap_values = xgb_explainer.shap_values(X_test)
xgb_shaps = xgb_explainer(X_test)

### Overall view
To give an initial glance of the effect each feature has on the models, we can plot out the summary plot from the SHAP library.
These plots are gathered from each individual instance.
The produced plot sorts the features in descending order of the contribution
 it has on the classifications.

It's clear that the alcohol level is the major contributor in both the models and the top 5 attributes generally agree with eachother, but the order and magnitude of their contribution differs between the models.

In [ ]:
shap.summary_plot(lr_shap_values, X_test, feature_names=wine.columns)

In [ ]:
shap.summary_plot(xgb_shap_values, X_test, feature_names=wine.columns)

The barplot also provides the average information of the contribution.

In [ ]:
shap.plots.bar(lr_shaps)

If we divide into cohorts it provides some additional information. The features are divided into the groups where the most prominent feature contributes the most.

So, when for an instance which has alcohol over 10.775 the features contrubtions are available in the red and white bars.

In [ ]:
shap.plots.bar(lr_shaps.cohorts(2).abs.mean(0))

In [ ]:
shap.plots.bar(xgb_shaps.cohorts(2).abs.mean(0))

### Individual instances
When we instead look at an individual data instance and its contribution to the prediction, we have a lot of options to visualize with. In all cells below there are various graphs that visualize the same things, you can use whichever you prefer for your visual aid later on.

In [ ]:
shap.initjs()
ind = 10
shap.force_plot(
    lr_explainer.expected_value, lr_shap_values[ind], X_test[ind,:],
    feature_names=wine.columns[:11]
)

In [ ]:
shap.plots.waterfall(lr_shaps[ind])

In [ ]:
shap.decision_plot(lr_explainer.expected_value, lr_shap_values[10], wine.columns[:11])

# we can also view multiple instances at once. Comment out below to view.
#shap.decision_plot(lr_explainer.expected_value, lr_shap_values[[6,8,10]], wine.columns[:11])

In [ ]:
shap.initjs()
shap.force_plot(
    xgb_explainer.expected_value, xgb_shap_values[ind], X_test[ind,:],
    feature_names=wine.columns[:11]
)

In [ ]:
xgb_shaps = xgb_explainer(X_test)
shap.plots.waterfall(xgb_shaps[ind])

### Correlated Features
Using hierarchical clustering, we can view how features are correlated to each other. This is built into shap.

We can use this information to simplify, and perhaps better the model, as correlated features often divide the importance.

In [ ]:
clustering = shap.utils.hclust(X_test, Y_test)
shap.plots.bar(lr_shaps, clustering=clustering, clustering_cutoff=0.57) # A higher cut off value means a larger distance between the features. I.e., larger means more independant and less redundant.

We see that the density feature is correlated to alcohol. If we remove density, we are likely to increase the importance of the alcohol feature.

Below we remove the density feature and retrain the model. We then look into if the predictive performance has changed and we visualize the same instance as before using the SHAP library.

In [ ]:
cols = wine.drop('density', axis=1).columns
wine_without_density = wine[cols]


X_test_new = np.concatenate((X_test[:,:7], X_test[:,8:]), axis=1)
X_train_new = np.concatenate((X_train[:,:7], X_train[:,8:]), axis=1)

lr_new = LogisticRegression(max_iter=1000)
xgb_new = xgboost.XGBClassifier(n_estimators=500, max_depth=2)
lr_new.fit(X_train_new, Y_train)
xgb_new.fit(X_train_new, Y_train)
print(accuracy_score(lr_new.predict(X_test_new), Y_test))
print(accuracy_score(xgb_new.predict(X_test_new), Y_test))

cols = list(wine.columns[:7]) + list(wine.columns[8:])

mask_new = shap.maskers.Independent(X_train_new)
lr_explainer_new = shap.LinearExplainer(lr_new, mask_new, feature_names=cols)
lr_shap_values_new = lr_explainer_new.shap_values(X_test_new)
lr_shaps_new = lr_explainer_new(X_test_new)

xgb_explainer_new = shap.TreeExplainer(xgb_new, X_train_new, feature_names=cols)
xgb_shap_values_new = xgb_explainer_new.shap_values(X_test_new)
xgb_shaps_new = xgb_explainer_new(X_test_new)

In [ ]:
shap.decision_plot(lr_explainer_new.expected_value, lr_shap_values_new[ind], feature_names=cols[:-1])

In [ ]:
shap.decision_plot(xgb_explainer_new.expected_value, xgb_shap_values_new[ind], feature_names=cols[:-1])

There are also additional functions you can look into at the [shap documentation](https://shap.readthedocs.io/en/latest/api.html) and examples via their [github page](https://github.com/shap/shap).

## LIME

Here we use LIME to get an interpretable model from our previously trained models.

In [ ]:
lime_explainer = lime.lime_tabular.LimeTabularExplainer(X_train ,class_names=['Low Quality', 'High Quality'], feature_names = wine.columns[:11],
                                                   #categorical_features=categorical_features,
                                                   #categorical_names=categorical_names,
                                                   kernel_width=3, verbose=False)

Using the same data instance as before, we show the contribution of each feature for the prediction. This example limits the number of features to 5. Try and see what happens if you increase/decrease the number of features to include.

In [ ]:
ind = 10
exp = lime_explainer.explain_instance(X_test[ind], lr.predict_proba, num_features=5)
exp.show_in_notebook()

We can also grab the rules we get from the interpretable model LIME creates.

In [ ]:
exp.as_list()

The code below just grabs the data instance and its corresponding feature values for you to compare against the rules above.

In [ ]:
list(zip(X_test[ind], wine.columns))

There are also more functions in LIME that you can look into through the [LIME documentation](https://lime-ml.readthedocs.io/en/latest/) and examples via the [github page](https://github.com/marcotcr/lime).

# Assignment

Here you are to apply the techniques from above to try and explain the output produced by a model.

Think about questions such as, but not limited to:
- Which parts of the data are mostly influencing the models output?
- Are some parts unecessary of the data?
- What can you do to improve explainability of the model?
- Etc.

Investigate, analyze, and possibly improve the model using the SHAP and LIME modules. Explain and show your steps as you go along.

## Load the data, create and train your model

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from joblib import load, dump
import pickle

# Import dataset
scaler = MinMaxScaler()
dataset = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(dataset['data'], dataset['target'], test_size=0.2, random_state=42)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Creating and training the model
clf = MLPClassifier(hidden_layer_sizes = [100,100], alpha = 5, solver = 'lbfgs', max_iter = 1000).fit(X_train_scaled, y_train)

print('Training score:', clf.score(X_train_scaled, y_train))
print('Testing score:', clf.score(X_test_scaled, y_test))

# Output should be
#Training score: 0.9802197802197802
#Testing score: 0.9736842105263158

## Students code and text below


The `MLPClassifier` could be considered a deep learning approach, however, the `DeepLearningExplainer` only supports `tensorflow` and `pytorch`. Hench the `KernelExplainer` was used here as it was a model agnostic approach. The only disadvantage it was slower.

In [ ]:
model_explainer = shap.explainers.KernelExplainer(
    lambda x: clf.predict_proba(x)[..., 0], 
    X_train_scaled, 
    seed=0
)
model_shaps = model_explainer(X_test_scaled)
model_shaps.feature_names = dataset["feature_names"]

Checking the SHAP results to see which features move the predictions more on the entire dataset.

In [ ]:
shap.plots.bar(model_shaps, max_display=model_shaps.data.shape[-1])

Running the model on different features based on their shapley values. The model was be iterated one feature at a time to see how the performance changes.

The model reached its stable result at 4 features for the test set. Though the prediction fluctuates from time to time on a single item, making the best model at 19 and 20 featuers.

In [ ]:
pruned_feature_indices = np.argsort(np.abs(model_shaps.values).mean(0))[::-1]
pruned_feature_scores = []

for n_features, _ in enumerate(pruned_feature_indices, 1):
    pruned_X_train = X_train_scaled[:, pruned_feature_indices[:n_features]]
    pruned_X_test = X_test_scaled[:, pruned_feature_indices[:n_features]]

    pruned_clf = MLPClassifier(
        hidden_layer_sizes = [100,100], 
        alpha = 5, 
        solver = 'lbfgs', 
        max_iter = 1000
    ).fit(pruned_X_train, y_train)
    pruned_feature_scores.append(pruned_clf.score(pruned_X_test, y_test))


In [ ]:
plt.plot(pruned_feature_scores)
plt.xlabel("N Features")
plt.ylabel("Model Score")
plt.title("N Features vs Model Score")
plt.show()

Checking the correlated features and see how they impact the model.

In [ ]:
clustering = shap.utils.hclust(X_test_scaled, y_test)

In [ ]:
shap.plots.bar(
    model_shaps, 
    clustering=clustering, 
    clustering_cutoff=0.1,
    max_display=model_shaps.data.shape[-1]
) 

Running the model on the uncorrelated features had the same performance as the base model. In this case it means one can focus on these uncorrelated features to save performance and reduce model complexity.

In [ ]:
corr_features = {
    "worst concave points",
    "worst radius",
    "mean concave points",
    "worst texture",
    "worst concavity",
    "mean texture",
    "worst smoothness",
    "worst symmetry",
    "radius error",
    "mean fractal dimension",
    "area error",
    "compactness error",
    "mean fractal dimension",
    "mean compactness",
}
corr_feature_indices = np.asarray([
    idx for idx, n in enumerate(dataset["feature_names"]) 
    if n in corr_features
])
corr_X_train = X_train_scaled[:, corr_feature_indices]
corr_X_test = X_test_scaled[:, corr_feature_indices]

corr_clf = MLPClassifier(
    hidden_layer_sizes = [100,100], 
    alpha = 5, 
    solver = 'lbfgs',
    max_iter = 1000
).fit(corr_X_train, y_train)

print('Training score:', corr_clf.score(corr_X_train, y_train))
print('Testing score:', corr_clf.score(corr_X_test, y_test))


Check the summary plot to see the impact of the features to shapley values for each sample. This should provide an idea how the model generates its predictions on a general level.

In [ ]:
shap.summary_plot(model_shaps.values, X_test_scaled, feature_names=dataset["feature_names"])

In general, the summary plot shows that the features are high on one side and low on the other side of the shapley values. This means that the features used by the model are highly separable and are linear.

The linear model performed well on the test set initial model. The only disadvantage was missing 5 data points on the train set. However, there was also an advantage that this model is more interpretable and could be easily understood while only giving up a small amount of performance. The simplicity of the model also means it was more efficient computationally.

In [ ]:
linear_clf = LogisticRegression().fit(X_train_scaled, y_train)

print('Training score:', linear_clf.score(X_train_scaled, y_train))
print('Testing score:', linear_clf.score(X_test_scaled, y_test))

A decision tree ensemble was also tested and also performed well though there are signs of overfitting. It missing a prediction compared to the `MLPClassifier` on the test set while also being more explainable compared to it. Though this adds more complexity compared to the linear model.

In [ ]:
tree_clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=5,
    random_state=0
).fit(X_train_scaled, y_train)

print('Training score:', tree_clf.score(X_train_scaled, y_train))
print('Testing score:', tree_clf.score(X_test_scaled, y_test))